In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [2]:
device = torch.device('cpu')
device

device(type='cpu')

In [3]:
model_name = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, output_hidden_states=True)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
from tqdm import tqdm

### Грузим посчитанные до этого эмбединги

In [5]:

# import pickle
# #with open("book_embeddings512.pkl", "rb") as f:
# #    embeddings_512 = pickle.load(f)
# # with open("book_embeddings256x.pkl", "rb") as f:
# #     embeddings256x = pickle.load(f)
# # with open("book_embeddings128x.pkl", "rb") as f:
# #     embeddings128x = pickle.load(f)
# with open("book_embeddings64x.pkl", "rb") as f:
#     embeddings64x = pickle.load(f)

In [6]:
# with open("book_embeddings512.pkl", "rb") as f:
#     embeddings_512 = pickle.load(f)
# with open("book_embeddings256x.pkl", "rb") as f:
#     embeddings_256 = pickle.load(f)


In [10]:
with open("book_embeddings256.pkl", "rb") as f:
    embeddings_256 = pickle.load(f)
with open("book_embeddings64x.pkl", "rb") as f:
    embeddings_64x = pickle.load(f)

EOFError: Ran out of input

#### Загружаю датасет, Обучаю эмбединг

In [7]:
books_new = pd.read_csv('parsing/all+++.csv')
annot_new = books_new['annotation']
books_new.shape



(32214, 6)

In [10]:
embeddings_256xxx = []

embeddings = [embeddings_256xxx] 
for i in range(0,len(embeddings)):
    
    if i == 0:
        length = 256
    embedding =embeddings[i]
    for annotation in tqdm(annot_new):
        annotation_tokens = tokenizer.encode_plus(
            annotation,
            add_special_tokens=True,
            max_length=length, # Ограничение на максимальную длину входной последовательности
            pad_to_max_length=True, # Дополним последовательность нулями до максимальной длины если слов меньше заданной длины
            return_tensors='pt' # Вернём тензоры PyTorch
        )

        with torch.no_grad():
            # Получим эмбеддинг аннотации
            outputs = model(**annotation_tokens)
            hidden_states = outputs.hidden_states
            last_hidden_state = hidden_states[-1][:,0,:] # Используем предпоследний слой для эмбеддинга
            last_hidden_state = torch.nn.functional.normalize(last_hidden_state)

            embedding.append((last_hidden_state).squeeze())

100%|█████████████████████████████████████| 32214/32214 [15:01<00:00, 35.74it/s]


#### Смотрю рекомендации 

In [28]:

queries = [
"Если бы ты был напитком, то каким?",
"Как успехи?",
"Детектив, где главная героиня - маленькая девочка сирота",
"Романтическая комедия с хорошим концом",
"Легкая биллетристика",
"Роман, где действия происходят в отеле",
"Исследование эволюции человека: от приматов до Homo sapiens",
"Революционные открытия в генетике",
"Квантовая теория поля и фундаментальные взаимодействия в физике",
"Искусственный интеллект и машинное обучение: современные достижения и будущие перспективы",
"Экологические изменения и глобальное потепление: причины, последствия и возможные решения",
"Роль мозга в познании и сознании: современные подходы исследований",
"Геномное редактирование и его этические аспекты",
"Межпланетные путешествия и исследование космоса: вызовы и достижения",
"Нейропластичность и возможности ее использования в реабилитации и обучении",
"Исследование микробиома человека и его влияние на здоровье и болезни",
"Кризисы в квантовой физике: от парадоксов до новых теорий",
"Биотехнологии и их роль в медицине, сельском хозяйстве и промышленности",
"Исследование климатических изменений в прошлом: анализ археологических данных и климатических прокси",
"Теория относительности и космология: понимание структуры и эволюции Вселенной",
"Генетическая инженерия и создание трансгенных организмов: перспективы и риски",
"Нейронаука и понимание мозговых расстройств: от депрессии до болезни Альцгеймера",
"Робототехника и автономные системы: достижения и применение в различных отраслях",
"Исследование квантовых компьютеров и их потенциал в решении сложных задач",
"Биологическое разнообразие и сохранение экосистем: вызовы и стратегии",
"Новые фронтиры в физике высоких энергий: открытие элементарных частиц и теории струн",
"Книга с многогранными персонажами, захватывающим сюжетом и глубокими философскими мыслями",
"Роман о тайной организации, в которую вовлечен главный герой, и смертельной опасности, которая грозит всему миру",
"Приключенческая сказка о потерянном сокровище, путешествиях по неизведанным землям и искушениях дружбы и предательства",
"Детективный триллер с запутанными загадками, интригами на высших уровнях и неожиданными разоблачениями",
"Научно-фантастическая эпопея о битве между разумными расами в далеком галактическом будущем, где ставки высоки и ничего не так, как кажется",
"Исторический роман о великом правителе, его страстной любви и трагической судьбе, разыгравшейся на фоне политических интриг и кровопролитных войн",
"Философская сага о смысле жизни, пути самопознания и стремлении к духовному просветлению",
"Психологический триллер с глубокой проникновенностью в психику главных героев, которые борются с внутренними демонами и секретами",
"Книга о жизненных испытаниях, внутренней силе и истинной значимости каждого мгновения",
"Фэнтезийная сага о мире, где магия, войны и политические интриги переплетаются, а судьба всего мира зависит от смелого героя",
"Книга про загадочное исчезновение с увлекательным сюжетом и неожиданными поворотами",
"Роман о запретной любви с яркими персонажами и эмоциональной глубиной",
"Приключенческая книга о потерянном мире, полном опасностей и загадок",
"Детектив с головоломками, запутанными интригами и непредсказуемым развитием событий",
"Научно-фантастический роман о будущем, где искусственный интеллект стал угрозой для человечества, с захватывающей атмосферой и красочными описаниями",
"Историческая сага о предательстве, борьбе за власть и страстных романтических отношениях в эпоху перемен",
"Философский роман о смысле жизни, размышлениях о судьбе и нахождении своего места в мире",
"Психологический триллер с загадками, скрытыми тайнами и атмосферой напряжения, которая заставляет держать вас в напряжении до последней страницы",
"Книга о восхождении главного героя, где вы сможете пройти вместе с ним через трудности и испытания, ощутить его триумф и познать смысл его усилий",
"Фэнтези-эпопея о противостоянии добра и зла, с яркими мирами, сложными персонажами и захватывающими сражениями"
          ]
#embeddings = [embeddings64x, embeddings_64xxx, embeddings_128xxx, embeddings_256xxx, embeddings_256, embeddings_512]          
embeddings = [embeddings64x, embeddings_256, embeddings_256xxx]
for query in queries:
    print(f'\n{query}')
    for i in range(0,len(embeddings)): # Если используешь 1 эмбединг, то заккоментируй отсюда
        if i == 0:
            length = 64
        else:
            length = 256
        # if i == 0:
        #     length = 64
        # elif i == 1:
        #     length = 64
        # elif i == 2:
        #     length = 128
        # elif i == 3:
        #     length = 256
        # elif i == 4:
        #     length = 256
        # else:
        #     length = 512
            
        print(f'\n{i+1}')
        print(f'\n{length}')
        
        embedding =embeddings[i] # ДО СЮДА
    
        
        query_tokens = tokenizer.encode_plus(
            query,
            add_special_tokens=True,
            max_length=length, # Здесь вставь правильную длину !!!!!!!!!!!!!!!! 
            pad_to_max_length=True, # Дополним последовательность нулями до максимальной длины
            return_tensors='pt' # Вернём тензоры PyTorch
        )
        with torch.no_grad():
            query_outputs = model(**query_tokens)
            query_hidden_states = query_outputs.hidden_states
            query_last_hidden_state = query_hidden_states[-1][:,0,:] # Используем предпоследний слой для эмбеддинга
            query_embedding = query_last_hidden_state.squeeze()
        cosine_similarities = torch.nn.functional.cosine_similarity(
        query_embedding.unsqueeze(0),
        torch.stack(embedding) # И здесь вставь правильный эмбединг!!!!!!!
        )

        cosine_similarities = cosine_similarities.numpy()

        indices = np.argsort(cosine_similarities)[::-1] # Сортировка по убыванию
        #rec_titles = []
        
        for i in indices[:3]:
            print(books_new['title'][i],books_new['page_url'][i]) 


Если бы ты был напитком, то каким?

1

64
Мастер дальних дорог https://www.biblio-globus.ru/product/10879316
Простые ответы на сложные вопросы https://www.biblio-globus.ru/product/10887037
Проклятая https://www.biblio-globus.ru/product/10860447

2

256
Сказки о гномах https://www.biblio-globus.ru/product/10843432
Приключения Славки Щукина.33 истории про вранье https://www.biblio-globus.ru/product/10887938
Вещие сестрички https://www.biblio-globus.ru/product/8981122

3

256
Несмертельные проклятья. Личная помощница ректора (Несмертельные проклятья #2) https://www.biblio-globus.ru/product/10873236
Наследник по контракту https://www.biblio-globus.ru/product/10886219
Энциклопедия сказочной доброты https://www.biblio-globus.ru/product/10861364

Как успехи?

1

64
42 истории для менеджера, или Сказки на ночь от Генри Минцберга https://www.biblio-globus.ru/product/10606015
Беременность. Современные лайфхаки доказательной медицины https://www.biblio-globus.ru/product/10906749
Бизнес. Пособие 

#### Сохраняю эмбединги

In [13]:
import pickle
with open("book_embeddings64xxx.pkl", "wb") as f:
    pickle.dump(embeddings_64xxx, f)

In [14]:
import pickle
with open("book_embeddings128xxx.pkl", "wb") as f:
    pickle.dump(embeddings_128xxx, f)

In [15]:
import pickle
with open("book_embeddings256xxx.pkl", "wb") as f:
    pickle.dump(embeddings_256xxx, f)

In [190]:
import pickle
with open("book_embeddings512.pkl", "wb") as f:
    pickle.dump(embeddings_512, f)